# LAB: Estimando hiperparámetros con `GridSearchCV` para Regresión Logística y KNN

## Introducción

El objetivo de esta práctica es que ajusten hiperparámetros usando Cross Validation. Para eso, usaremos `GridSearchCV`.

Utilizaremos el dataset ya trabajado sobre cáncer de mama. Contiene información de estudios clínicos y celulares. El objetivo es predecir el carácter benigno ($class_t=0$) maligno ($class_t=1$) del cáncer en función de una serie de predictores a nivel celular.

    + class_t es la variable target
    + el resto son variables con valores normalizados de 1 a 10

[Aquí](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names) pueden encontrar más información sobre el dataset.

**Nota:** se eliminaron del dataset original 16 casos con valores perdidos en algunos campos.

## Tareas

Para esta práctica deberá 

1. Construir dos clasificadores: Regresión Logística y K-Vecinos más Cercanos (KNN)
2. Estimar los hiperpáremtros del modelo

          *2.1 LogReg:* deberá tunear un modelo con solver 'saga', C's = 1, 10, 100, 1000, y regularización L1 y L2
          
          *2.2 KNN:* deberá tunear tanto el parámetro k, como la medida del peso dado a los K vecinos (uniforme o distancia). También podría probar con el parámetro p que define el tipo de distancia con el que se calculan los vecinos más cercanos.
      
3. Estimar los modelos finales
4. Evaluar cuál de los dos performa mejor

**Importante:** recuerde que deberá diseñar cuidadosamente las diferentes estrategias de validación de las diferentes etapas de estimación del modelo.

Importamos los paquetes necesarios


In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import normalize

Importamos el dataset

In [2]:
df = pd.read_csv('../Data/breast-cancer.csv', header = None)
df.columns = ['ID', 'clump_Thickness', 'unif_cell_size', 'unif_cell_shape', 'adhesion', 'epith_cell_Size', 'bare_nuclei',
              'bland_chromatin ','norm_nucleoli', 'mitoses', 'class_t']

In [3]:
df.shape

(683, 11)

In [4]:
df.head()

,ID,clump_Thickness,unif_cell_size,unif_cell_shape,adhesion,epith_cell_Size,bare_nuclei,bland_chromatin,norm_nucleoli,mitoses,class_t
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


Recodificamos las clases en "0" y "1"

In [5]:
df.class_t[df['class_t'] == 2] = 0
df.class_t[df['class_t'] == 4] = 1

Hacemos el split entre target y features

**Pista 1:** Conviene realizar dos listas, una con los estimadores de los modelos y otra con la grid de parámetros a estimar en cada modelo.

**Pista 2:** Conviene iterar sobre esas listas para estimar los hiperparámetros de los modelos

## Diferencia de performance entre Random Search y Gridsearch

Dado el siguiente conjunto de parámetros:

        param_dist = {
                    'n_neighbors': range(1,200),
                    'weights' : ['uniform', 'distance'],
                    'p' : [1, 2, 3]
                    }

Implementar una búsqueda del conjunto óptimo de hiperparámetros tanto con GridSearchCV como con RandomSearchCV.
Verificar la diferencia en cada caso de:
    
    1. El tiempo de ejecución (utilizando la biblioteca time)
    2. La combinación óptima de parámetros
    3. La performance del mejor modelo en cada caso sobre los datos del test set que separamos anteriormente en términos de accuracy


In [6]:
from sklearn.model_selection import RandomizedSearchCV

In [7]:
def busquedaGridsearch(params_):
    gs = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=params_, scoring='accuracy', cv=10, n_jobs=4)
    fit = gs.fit(X_train, y_train)
    return gs    

In [8]:
def busquedaRandomSearch(params_,iter_):
    gs = RandomizedSearchCV(estimator=KNeighborsClassifier(), param_distributions=params_, scoring='accuracy', cv=10, n_jobs=4, n_iter = iter_ )
    fit = gs.fit(X_train, y_train)
    return gs    

In [9]:
from sklearn.metrics import accuracy_score

def obtener_performance(estimator):
    y_pred = estimator.predict(X_test)
    return accuracy_score(y_pred,y_test, normalize = True)